In [ ]:
import pathlib
import seaborn as sb
import networkx
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from utils.paths import slugify
from utils.plotting import threshold_plot, plot_interval

## Declare parameters to use throughout

In [ ]:
events = [(12 * i, '', 'dashed') for i in range(4 + 1)]
phases = [
    (0, 12, 'Day 1'),
    (12, 24, 'Night 1'),
    (24, 36, 'Day 2'),
    (36, 48, 'Night 2')
]

output_directory = None

### Load temporal data

In [ ]:
node_table_filepath = '../data/temporal_data/circadian/circadian_temporal_node_data_24_hour_genes_mean_48.csv'
static_network_filepath = '../data/static_networks/circadian_net.edgelist'

thresholds = [round(0 + (0.1 * i), 1) for i in range(10)]
combine_node_weights = lambda x, y: x * y

node_table = pd.read_csv(node_table_filepath, sep='\t', index_col=0)
static_network = networkx.read_edgelist(static_network_filepath)

edge_table = pd.DataFrame()
for i in node_table.columns.values:
    for j in node_table.columns.values:
        if i < j and static_network.has_edge(i, j):
            edge_table[f'({i}, {j})'] = combine_node_weights(node_table[i], node_table[j])

### Normalise (if data is not already normalised)

In [ ]:
for node in node_table.columns.values:
    node_min = node_table[node].min()
    difference = node_table[node].max() - node_min
    node_table[node] = (node_table[node] - node_min) / difference

for edge in edge_table.columns.values:
    edge_min = edge_table[edge].min()
    difference = edge_table[edge].max() - edge_min
    edge_table[edge] = (edge_table[edge] - edge_min) / difference

number_of_nodes = node_table.shape[1]
number_of_edges = edge_table.shape[1]
times = node_table.index.values

### Plot node activities

In [ ]:
fig, ax = plt.subplots(figsize=(9, 6))
cmap = cm.get_cmap('rainbow')

colour_interval = 1.0 / number_of_nodes
for j, node in enumerate(node_table.columns.values):
    color = cmap(colour_interval * j)
    ax.plot(times, node_table[node].values, color=color)

title = f'Node activities: {slugify(pathlib.Path(node_table_filepath).stem)}'
fig.suptitle(title, weight='bold')

### For each threshold, plot edge activities

In [ ]:
figsize = (9, 6*len(thresholds))
fig, axs = plt.subplots(len(thresholds), figsize=figsize)
cmap = cm.get_cmap('rainbow')

for ax, threshold in zip(axs, thresholds):
    colour_interval = 1.0 / number_of_edges

    for j, edge in enumerate(edge_table.columns.values):
        y = edge_table[edge].values
        color = cmap(colour_interval * j)

        if np.all(y >= threshold):
            ax.plot(times, y, color=color)
        elif np.all(y < threshold):
            ax.plot(times, y, color='k')
        else:
            threshold_plot(times, y, threshold, 'k', color, ax=ax)
        ax.axhline(threshold, color='k', ls='--')

    ax.set_ylim(0,1)
    ax.set_title(f'Thresholded edges, threshold={threshold}', weight='bold')

title = f'Edge activities: {slugify(pathlib.Path(node_table_filepath).stem)}'
fig.suptitle(title, weight='bold', y=0.90)

### Plot the binarized edges

In [ ]:
figsize = (9, 15 * len(thresholds))
fig, axs = plt.subplots(len(thresholds), figsize=figsize)

for ax, threshold in zip(axs, thresholds):
    for j, edge in enumerate(edge_table.columns.values):
        binarized_edges = ((edge_table[edge] > 0) & (edge_table[edge] >= threshold)).astype(int).values
        plot_interval(binarized_edges, times, y=number_of_edges-1-j, ax=ax, color='C0')

        # For readability, highlight one in every two lines
        if j % 2==0 :
            plot_interval(np.ones_like(binarized_edges), times, y=j, ax=ax, color='lightgrey', zorder=-3)

    ax.set_yticks([k + 0.25 for k in range(number_of_edges)])
    labels = list(edge_table.columns.values)[::-1]
    ax.set_yticklabels(labels, fontsize="xx-small")
    ax.set_ylim(0, number_of_edges)

    ax.set_xlim(min(times), max(times))
    ax.set_xticks(times)
    ax.set_xticklabels(times, rotation=45, ha='right', fontsize='x-small')
    ax.grid(axis='x')

    ax.set_title(f'Binarised edges, threshold={threshold}', weight='bold')
    sb.despine(left=True)

title = f'Binarized edges: {slugify(pathlib.Path(node_table_filepath).stem)}'
fig.suptitle(title, weight='bold', y=0.89)